In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
import joblib
import random
import gc

2025-07-01 12:59:49.014926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


In [3]:
df = pd.read_hdf("./dataset.h5", key="df")
df

,Cqt_C,Cqt_Db,Cqt_D,Cqt_Eb,Cqt_E,Cqt_F,Cqt_Gb,Cqt_G,Cqt_Ab,Cqt_A,Cqt_Bb,Cqt_B,chord
0,0.471546,0.222220,0.447305,0.591948,0.477934,1.000000,0.274915,0.219494,0.184126,0.101357,0.065740,0.100825,FMin7
1,0.471546,0.222220,0.447305,0.591948,0.477934,1.000000,0.274915,0.219494,0.184126,0.101357,0.065740,0.100825,FMin7
2,0.525008,0.214509,0.273993,0.357224,0.350424,1.000000,0.180266,0.187004,0.137805,0.085566,0.054410,0.103364,FMin7
3,0.548730,0.214509,0.273993,0.357224,0.350424,1.000000,0.180266,0.187004,0.137805,0.085566,0.054410,0.103438,FMin7
4,0.606835,0.214509,0.273993,0.357224,0.350424,1.000000,0.180266,0.187004,0.137805,0.085566,0.054410,0.113766,FMin7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,0.981889,0.063128,0.412929,0.034860,0.079848,0.029547,0.113578,0.030053,0.020699,0.045428,0.011384,0.068915,D7
1194,0.981889,0.064797,0.348677,0.036478,0.079848,0.033618,0.120837,0.045060,0.025011,0.039565,0.013037,0.068915,D7
1195,0.981889,0.063128,0.301315,0.036478,0.079848,0.042244,0.111068,0.045060,0.027699,0.037074,0.027852,0.067913,D7
1196,0.981889,0.064797,0.301315,0.054282,0.079848,0.043336,0.063070,0.046707,0.029380,0.033834,0.034683,0.067913,D7


In [4]:
print("Chords")
print(df["chord"].value_counts())

Chords
chord
CMin7     11630
AbMin7    11611
GbMin7    11600
DbMin7    11514
FMin7     11264
EMin7     11163
AMin7     11118
DMin7     11039
BbMin7    11026
EbMin7    10884
GMin7     10876
BMin7     10825
AMaj7      6860
EMaj7      6518
GbMaj7     6493
CMaj7      6406
AbMaj7     6379
FMaj7      6278
BbMaj7     6270
GMaj7      6249
DbMaj7     6240
DMaj7      6212
EbMaj7     6185
BMaj7      6135
G7         2692
B7         2536
Db7        2428
Bb7        2427
Ab7        2426
C7         2400
D7         2399
Eb7        2389
A7         2384
Gb7        2359
E7         2345
F7         2340
Name: count, dtype: int64


In [5]:
y = df["chord"]
X = df.drop(columns="chord")
encoder = sk.preprocessing.LabelEncoder()
y_encoded = encoder.fit_transform(y)

joblib.dump(encoder, "./encoder.xz")

['./encoder.xz']

In [6]:
del df
gc.collect()

20

# LSTM

In [7]:
SEQUENCE_LEN = 20 # 0.1 * 20.0 = 2 sec of sequence data

X_seq, y_encoded_seq = None, None
X_seq_list = []
y_encoded_seq_list = []
for i in range(len(X) - SEQUENCE_LEN + 1):
    X_seq_list.append(X.values[i : i + SEQUENCE_LEN, :])
    y_encoded_seq_list.append(y_encoded[i + SEQUENCE_LEN - 1])

X_seq, y_encoded_seq = np.array(X_seq_list), np.array(y_encoded_seq_list)

print("X sequence shape: ", X_seq.shape)
print("y sequence shape: ", y_encoded_seq.shape)

del X_seq_list
del y_encoded_seq_list
gc.collect()

X sequence shape:  (239881, 20, 12)
y sequence shape:  (239881,)


0

In [8]:
X_seq_flat = X_seq.reshape(X_seq.shape[0], -1)
_, unique_idx = np.unique(X_seq_flat, axis=0, return_index=True)
unique_idx = np.sort(unique_idx)

X_seq = X_seq[unique_idx]
y_encoded_seq = y_encoded_seq[unique_idx]

print("X deduplicate shape: ", X_seq.shape)
print("y deuplicate shape:  ", y_encoded_seq.shape)

del X_seq_flat
gc.collect()

X deduplicate shape:  (239881, 20, 12)
y deuplicate shape:   (239881,)


0

In [9]:
X_seq_train, X_seq_test, y_seq_train, y_seq_test = sk.model_selection.train_test_split(
    X_seq,
    y_encoded_seq,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded_seq,
)

print(f"X_train: {len(X_seq_train)}")
print(f"y_train: {len(y_seq_train)}")
print(f"X_test:  {len(X_seq_test)}")
print(f"y_test:  {len(y_seq_test)}")

del X_seq
del y_encoded_seq
gc.collect()

X_train: 191904
y_train: 191904
X_test:  47977
y_test:  47977


0

In [10]:
print("Total feature: ", X_seq_train.shape[2])
print("Total class:   ", len(encoder.classes_))

Total feature:  12
Total class:    36


In [11]:
class_weights = sk.utils.class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_seq_train),
    y=y_seq_train,
)
class_weight_dict = dict(enumerate(class_weights))

In [12]:
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(SEQUENCE_LEN, X_seq_train.shape[2]))),
    tf.keras.layers.Dropout(0.25),
        
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Dense(len(encoder.classes_), activation='softmax'),
])

model_lstm.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model_lstm.summary()

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True,
)

history = model_lstm.fit(
    X_seq_train,
    y_seq_train,
    epochs=500,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping],
    class_weight=class_weight_dict,
)

Epoch 1/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 62s 12ms/step - accuracy: 0.3224 - loss: 2.3488 - val_accuracy: 0.5043 - val_loss: 1.6895
Epoch 2/500
4256/4798 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5081 - loss: 1.6268

In [ ]:
model_lstm.evaluate(X_seq_test, y_seq_test)
y_pred = np.argmax(model_lstm.predict(X_seq_test), axis=1)
print(
  sk.metrics.classification_report(y_seq_test, y_pred, target_names=encoder.classes_)
)

In [ ]:
model_lstm.save("model_lstm.keras")
pd.DataFrame(history.history).to_csv("history.csv", index=False)